## Select the variables for the behaviour of the script

* *batch_size_used*: Defines the batch size used by the dataloader.
* *percentage_used*: Defines the percentage of the total dataset that will be used.
* *folder_name*: Name of the folder where the training results will be saved.
* *net_name*: Identifier of the network.
* *n_nets*: Number of networks that will be trained.
* *n_epochs*: Maximum number of epochs, early stopping is activated.
* *test_interval*: Defines how many times the network will calculate the MAE and RMSE. 

In [ ]:
batch_size_used = 1000 # Has to be fixed
percentage_used = 100
folder_name = "Conv1_MDN"
net_name = "Conv13"
n_epochs =  100 # Has to be fixed
n_nets = 3
test_interval=5 

## Import the required libraries

In [ ]:
# import libraries
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import shutil
import re
import math
import time
import glob
import gzip
import sys
import matplotlib.pyplot as plt
import os
import random
import h5py


## Declare and initialize the dataloader
1. Declare the class EventsData, which will act as the model's dataloader.
2. Declare the function that will initialize the three dataloader.
3. Initialize the three dataloaders

In [ ]:
class EventsData(Dataset):
    def __init__(self,data_dir,per=100,batch_size=500):
        # Save the directory of the data
        self.data_dir = data_dir
        
        # Get the names of the files
        self.names=glob.glob(str(data_dir)+'*.hdf5')
        # Number of files in the dir
        self.size_dir=len(self.names)
        
        # Set the files acording to the percentage
        self.size_dir=math.ceil(len(self.names)*(per/100))
        self.names=self.names[0:self.size_dir]
        #random.shuffle(self.names)
        
        # Get the number of events per file
        print(self.names[1])
        f = h5py.File(self.names[0],'r')
        self.size_file=f['y'].shape[0]
        
        # Get the total number of events
        self.total_events=0
        for name in self.names:
            f = h5py.File(name,'r')
            y = f['y']
            self.total_events+=y.shape[0]
            
        # Load the whole dataset into the RAM
        self.data_big = torch.zeros(self.total_events,25,161)
        self.target_big = torch.zeros(self.total_events)

        print("Reading "+str(self.data_dir)+" with "+str(self.size_dir)+" files.")
        for a in range(len(self.names)):
            f = h5py.File(self.names[a],'r')
            self.data_big[(a*1000):(((a+1)*1000))]=torch.tensor(f['X1'][:,:,:,0])
            self.target_big[(a*1000):(((a+1)*1000))]=torch.tensor(f['y'][:,0])
            self.target_big[(a*1000):(((a+1)*1000))].size()
            
        # Number of iterations to finish the dataset
        self.batch_size=batch_size
        self.iters=math.floor(self.total_events/batch_size)
        self.iters_per_file= math.floor(self.size_file/batch_size)
        self.real_events=self.batch_size*self.iters

        print("There are "+str(self.total_events)+" events.")
        print("In "+str(self.size_dir)+" separate files.")
        print("Each file containing "+str(self.size_file)+" events.")
        print("In "+str(self.iters)+" iterations")
        print("The real number of events is: "+str(self.real_events))
        


    def get_len(self):
        return self.real_events
    
    def get_iter(self):
        # Returns the number of iteracions og getitem to finish the dataset
        return self.iters

    def get_batch(self, idx):
        # Get the file that shall be opened
        ind1=idx*self.batch_size
        ind2=((idx+1)*self.batch_size)
        
        #print(ind1)
        #print(ind2)
        
        data=self.data_big[ind1:ind2,:,:]
        target=self.target_big[ind1:ind2]
        
        # Get the events that will be extracted from the file
        #ind2=ind+self.batch_size
        
        data=data.unsqueeze(dim=3)
        target=target.unsqueeze(dim=1)
        
        #Only for conv with modulus
        data=data.transpose(1,3)
        data=data.transpose(2,3)
        
        
        #target[:,0]=torch.tensor(f['y'][ind:ind2,0])
        target=torch.arccos(target)
    
        
        return data.float(),target.float()

In [ ]:
def init_data(percentage,batch):
        # Save for exporting
        percentage=percentage
        batch_size=batch
        # Initialize the datasets
        print("Train dataset:")
        training_data = EventsData(data_dir='Mod_full_dist/train_data_fixed/', per=percentage, batch_size=batch);
        print()

        print("Validation dataset:")
        validation_data=EventsData(data_dir='Mod_full_dist/validation_data_fixed/', per=percentage, batch_size=batch);
        print()

        print("Test dataset:")
        test_data=EventsData(data_dir='Mod_full_dist/test_data_fixed/', per=percentage, batch_size=batch);
        print()

        return training_data,validation_data,test_data

In [ ]:
train,val,test = init_data(percentage_used,batch_size_used)

## Declare the model's architecture

In [ ]:
# This class will contain the NN architecture, it will be pushed to the GPU
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        kernel_size=(5,5)
        
        #conv layer, sees 25x161x1 tensor
        self.conv1 = nn.Conv2d(1, 16, kernel_size,padding=1)
        self.conv2 = nn.Conv2d(16, 36, kernel_size,padding=1)
        

        
        self.pool = nn.MaxPool2d((2, 2))
        input_flatten = 5472
        hidden_1 = 500
        hidden_2 = 200
        hidden_3 = 50
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(input_flatten, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.mu = nn.Linear(hidden_3, 1)
        self.sigma = nn.Linear(hidden_3,1)
        
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.25)


    def forward(self, x):
        input_flatten = 5472
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
   
        x = x.reshape(-1, input_flatten)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        mu = self.mu(x)
        sigma = torch.exp(self.sigma(x))
        return mu,sigma

## Define the custom loss and the Net_Info class

The Net_Info class will run the training of the model, saving all the information for a more convient control.

In [ ]:
# Nueva función de coste
def custom_loss(target,mu,sigma):
    # Create the normal distribution
    #print(mu[1])
    dist = torch.distributions.Normal(loc=mu, scale=sigma)
    # Obtain the -PDF and reduce it to the mean
    # Shall return a real number only
    loss = torch.mean(-dist.log_prob(target))
    return loss

In [ ]:
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

# This class will contain a NN model and all of its functions and data 
# It won't be pushed to the GPU
class Net_Info():
    # Constructor for the predictor
    # Data regarding the NN shall be passed here
    def __init__(self,model,folder,name):
        # Route to save the model
        self.folder=folder
        self.name=name

        # Set the optimizer and loss functions
        self.optimizer=optim.SGD(model.parameters(),lr=0.002)
        #self.loss_function=nn.L1Loss()
        
    # Function to set the model name and folder
    # Used to save the name in it
    def location(self,name,folder):
        self.folder=folder
        self.model_name=name

    def init_data(self,percentage,batch):
        # Save for exporting
        self.percentage=percentage
        self.batch_size=batch
        

    # Save all the data from the class, except the arquitecture of the net
    def save_params(self,model):
        # Open the text file
        file1=open(str(self.folder)+"/"+str(self.name)+"_params.txt","w+")
        file1.write("Data Info:\n")
        file1.write(str(self.percentage)+"\n"+str(self.batch_size)+"\n"+str(self.n_epochs)+"\n")
        file1.write("Last Val Info:\n")
        file1.write(str(self.valid_loss_min)+"\n")
        file1.write("Last Errors:\n")
        file1.write(str(self.MAE)+"\n"+ str(self.RMSE)+"\n")
        file1.write("Loss and Optimier\n")
        #file1.write(str(self.loss_function)+"\n")
        file1.write(str(self.optimizer))
        file1.close()
        file2=open(str(self.folder)+"/"+str(self.name)+"_architecture.txt","w+")
        file2.write(str(model))
        file2.close()

    def plot_loss(self):
        plt.figure(figsize=(10,10),dpi=800)
        plt.plot(self.train_loss_temp[:self.n_epochs],linewidth=3.0,color="r")
        plt.plot(self.valid_loss_temp[:self.n_epochs],linewidth=3.0,color="g")
        plt.title('{} | MAE = {:.3f} | RMSE = {:.3f}'.format(self.name,self.MAE,self.RMSE))
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.legend(['Training Loss','Validation Loss'], loc = 'upper left')
        # plt.show()
        plt.savefig(str(self.folder)+"/"+str(self.name)+"_loss.jpg")

    def plot_pred(self):
        plt.figure(figsize=(5,5),dpi=100)
        plt.scatter(self.target_temp,self.output_temp)
        plt.xlabel('True Values ')
        plt.ylabel('Predictions ')
        plt.axis('equal')
        plt.axis('square')
        plt.title("Target vs. Prediction values of "+str(self.name))
        plt.savefig(str(self.folder)+"/"+str(self.name)+"_scatter.jpg")
        plt.close();
      
        

    def train_model(self,net,n_epochs,test_inter,train,val,test,valid_loss_min=np.Inf):
        self.n_epochs = n_epochs
        # If valid loss is not inf, the load the model
        if valid_loss_min>1000:
            self.valid_loss_min = np.Inf # track change in validation loss
        else:
            self.valid_loss_min = valid_loss_min
            net.load_state_dict(torch.load(str(self.folder)+'/'+str(self.name)+'.pt'))
            

        self.train_loss_temp = np.zeros([n_epochs,1])
        self.valid_loss_temp = np.zeros([n_epochs,1])
        # If this number goes to zero, the training will stop
        last_save=10
        for epoch in range(1, n_epochs+1):
            # keep track of training and validation loss
            train_loss = 0.0
            valid_loss = 0.0



            # train the net #
            net.train()
            for batch in range(train.get_iter()):
                # Get the data
                data,target=train.get_batch(batch)
                # move tensors to GPU if CUDA is available
                data, target = data.cuda(), target.cuda()
                # clear the gradients of all optimized variables
                self.optimizer.zero_grad()
                # forward pass: compute predicted outputs by passing inputs to the net
                mu,sigma = net(data)
                # calculate the batch loss
                loss = custom_loss(target,mu,sigma)
                # backward pass: compute gradient of the loss with respect to net parameters
                loss.backward()
                # perform a single optimization step (parameter update)
                self.optimizer.step()
                # update training loss
                # and reboot if nan
                if math.isnan(loss.item()) == True:
                    net.apply(weight_reset)
                    print("Reseteo insacioso")
                train_loss += loss.item()


            # validate the net #
            net.eval()
            for batch in range(val.get_iter()):
                # Get the data
                data,target=val.get_batch(batch)   
                # move tensors to GPU if CUDA is available
                data, target = data.cuda(), target.cuda()
                # forward pass: compute predicted outputs by passing inputs to the net
                mu,sigma = net(data)
                # calculate the batch loss
                loss = custom_loss(target,mu,sigma)
                # update average validation loss 
                valid_loss += loss.item()
            #el data,target
            # calculate average losses
            train_loss = train_loss/train.get_len()
            valid_loss = valid_loss/val.get_len()

            # Append the losses to the historical ones
            self.train_loss_temp[epoch-1]=train_loss
            self.valid_loss_temp[epoch-1]=valid_loss            

            # print training/validation statistics 
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                epoch, train_loss, valid_loss))
            # Save the net if the loss goes down
            # Show also the % of error down
            if valid_loss <= self.valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving net ...'.format(
                self.valid_loss_min,
                valid_loss))
                torch.save(net.state_dict(), str(self.folder)+'/'+str(self.name)+'.pt')
                # Show by how much the validation loss has dropped
                valid_percent=((self.valid_loss_min-valid_loss)/self.valid_loss_min)*100
                print('Validation loss has dropped {:.2f}%'.format(valid_percent))
                self.valid_loss_min = valid_loss
                if valid_percent>=1:
                    last_save=15
                    print("Keep training")
                else:
                    last_save += 1
            else:
                last_save -= 1;
                
            # Show iterations to stop
            print('Training will stop in '+str(last_save)+" epochs ...")
            print("")
            
            # Check the test error and the 
            if (epoch%test_interval)==0:
                # self.target_temp=np.zeros([self.test_data.get_len(),1])
                # self.output_temp=np.zeros([self.test_data.get_len(),1])
                # Init the errors
                MAE=0
                MSE=0
                for batch in range(test.get_iter()):
                    # Get the data
                    data,target=test.get_batch(batch)
                    # move tensors to GPU if CUDA is available
                    data = data.cuda()
                    # Get the results from the foward pass to the CPU 
                    # And get it as an numpy matrix
                    mu,sigma = net(data)#.cpu().detach().numpy()
                    target=target.numpy()*(180.0/math.pi)
                    output = mu.cpu().detach().numpy()*(180.0/math.pi)
                    # calculate the batch loss
                    MAE += np.sum(np.abs(output-target))
                    MSE += np.sum((output-target)**2)
                    # print(output)
                    # Append to the historical value
                    # self.target_temp[(batch*self.batch_size):((batch+1)*self.batch_size)]=target
                    # self.output_temp[(batch*self.batch_size):((batch+1)*self.batch_size)]=output

                # self.plot_pred()
                # calculate average losses
                self.MAE = MAE/test.get_len()
                self.RMSE = np.sqrt(MSE/test.get_len())
                    
                print("The MAE is "+str(self.MAE))
                print("The RMSE is "+str(self.RMSE))

            # Check if the net is not progressing
            if last_save<=0:
                print("Early stopping")
                self.n_epochs=epoch
                break
        # Compute the test error for the best performing model
        # Load the model
        net.load_state_dict(torch.load(str(self.folder)+'/'+str(self.name)+'.pt'))
        MAE=0
        MSE=0
        for batch in range(test.get_iter()):
            # Get the data
            data,target=test.get_batch(batch)
            # move tensors to GPU if CUDA is available
            data = data.cuda()
            # Get the results from the foward pass to the CPU 
            # And get it as an numpy matrix
            mu,sigma = net(data)
            target=target.numpy()*(180.0/math.pi)
            output = mu.cpu().detach().numpy()*(180.0/math.pi)
            # calculate the batch loss
            MAE += np.sum(np.abs(output-target))
            MSE += np.sum((output-target)**2)
            # print(output)
            # Append to the historical value
            # self.target_temp[(batch*self.batch_size):((batch+1)*self.batch_size)]=target
            # self.output_temp[(batch*self.batch_size):((batch+1)*self.batch_size)]=output

        # self.plot_pred()
        # calculate average losses
        self.MAE = MAE/test.get_len()
        self.RMSE = np.sqrt(MSE/test.get_len())

        print("The MAE is "+str(self.MAE))
        print("The RMSE is "+str(self.RMSE))


## Train the models

In [ ]:
# Try to create the folder name
try:
    os.mkdir(str(folder_name))
except FileExistsError:
    print(str(folder_name)+ " directory already exists")

# Copy the modified python script with the net name into the subfolder
shutil.copy2(sys.argv[0],str(folder_name)+"/"+str(net_name)+".py")  

# Check if cuda is available and set as default device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using decive "+str(device))
# Run all the models
# Run all the learning rates

for b in range(n_nets):
    # initialize the NN
    new_name=str(net_name)+"_"+str(b)
    print("Running "+str(new_name))
    net=Net()
    print(net)
    net.cuda()
    # Initialize the Net_Info object
    Model=Net_Info(net,folder_name,new_name)
    Model.optimizer=optim.Adadelta(net.parameters())
    # Initialize the datasets
    Model.init_data(percentage_used,batch_size_used)
    # This next may be inside of a function
    Model.train_model(net,n_epochs,test_interval,train,val,test)
    # Save the model
    Model.save_params(net)
    Model.plot_loss() 
    
